In [1]:
loc_data = '/Users/yuxuan/Desktop/kaggle_optiver/optiver-realized-volatility-prediction'

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import os
from sklearn.metrics import r2_score
import glob
import matplotlib.pyplot as plt

In [ ]:
from helper import *

In [4]:


def realized_volatility_per_time_id(file_path, prediction_column_name):
    df_book_data = log_return_per_time_id(file_path)
    # compute realized volatility ONLY for HISTORICAL values
    df_realized_vol_per_stock = df_book_data.groupby('time_id')['log_return'].agg(realized_volatility).to_frame().reset_index()
    # pred is relatived volatility of log-return
    df_realized_vol_per_stock = df_realized_vol_per_stock.rename(columns = {'log_return':prediction_column_name})
    stock_id = file_path.split('=')[1]
    df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock['time_id'].apply(lambda x:f'{stock_id}-{x}')
    df_out = df_realized_vol_per_stock[['row_id',prediction_column_name]]
    return df_out

In [5]:
# Replace with a map-reduce style?
def past_realized_volatility_per_stock(list_file,prediction_column_name):
    df_past_realized = pd.DataFrame()
    for file in list_file:
        df_past_realized = pd.concat([df_past_realized,
                                     realized_volatility_per_time_id(file,prediction_column_name)])
    return df_past_realized

### Replicate naive prediction

In [3]:
list_order_book_file_train = glob.glob(f'{loc_data}/book_train.parquet/*')

In [5]:
file0 = list_order_book_file_train[0]

In [6]:
file0

'/Users/yuxuan/Desktop/kaggle_optiver/optiver-realized-volatility-prediction/book_train.parquet/stock_id=17'

In [77]:
df = realized_volatility_per_time_id(file0, 'pred')

In [51]:
df.head()

,row_id,pred
0,17-5,0.004091
1,17-11,0.002155
2,17-16,0.002566
3,17-31,0.002221
4,17-62,0.002155


In [53]:
df_past_realized_train = past_realized_volatility_per_stock(list_file=list_order_book_file_train,
                                                           prediction_column_name='pred')

In [54]:
train = pd.read_csv(f'{loc_data}/train.csv')

In [56]:
df_past_realized_train.head()

,row_id,pred
0,17-5,0.004091
1,17-11,0.002155
2,17-16,0.002566
3,17-31,0.002221
4,17-62,0.002155


In [67]:
def get_rowid(row):
    return f'{row.stock_id.astype(int)}-{row.time_id.astype(int)}'

In [69]:
train['row_id'] = train.apply(get_rowid, axis=1)
train = train[['row_id','target']]

In [70]:
train.head()

,row_id,target
0,0-5,0.004136
1,0-11,0.001445
2,0-16,0.002168
3,0-31,0.002195
4,0-62,0.001747


In [71]:
df_joined = train.merge(df_past_realized_train, on = ['row_id'], how = 'left')

In [72]:
df_joined.head()

,row_id,target,pred
0,0-5,0.004136,0.004499
1,0-11,0.001445,0.001204
2,0-16,0.002168,0.002369
3,0-31,0.002195,0.002574
4,0-62,0.001747,0.001894


The actual target is a forward return, while the prediction is based on backward return.

1. Timeid is not sequential, which implies that the longest lookback return we can compute is 10mins, as the length of a bucket is 10mins.
2. Variables from book_train.parquet not used in the demo: bid_price2, ask_price2, bid_size2, ask_size2.
3. Used variables from book_train.parquet are only used through WAP
4. trade_train.parquet is not used, which looks important

In [90]:
list_trade_file_train = glob.glob(f'{loc_data}/trade_train.parquet/*')

In [91]:
file0 = list_order_book_file_train[0]

In [83]:
df_book = log_return_per_time_id(file0)

In [84]:
df_book.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,wap,log_return
1,5,2,1.001186,1.001581,1.001016,1.001750,100,100,63,100,1.001383,0.000066
2,5,4,1.001186,1.001750,1.001016,1.002032,176,100,63,2,1.001546,0.000162
3,5,5,1.001355,1.001750,1.001186,1.001891,100,100,76,63,1.001552,0.000007
4,5,6,1.001355,1.001750,1.001186,1.002032,200,100,76,2,1.001618,0.000066
5,5,8,1.001355,1.001750,1.001186,1.002032,200,200,76,2,1.001552,-0.000066


In [89]:
df_book.seconds_in_bucket.sort_values().unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [92]:
df_trade_data = pd.read_parquet(file0)

In [93]:
df_trade_data

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,5,0,1.001186,1.001581,1.001044,1.001750,100,200,63,100
1,5,2,1.001186,1.001581,1.001016,1.001750,100,100,63,100
2,5,4,1.001186,1.001750,1.001016,1.002032,176,100,63,2
3,5,5,1.001355,1.001750,1.001186,1.001891,100,100,76,63
4,5,6,1.001355,1.001750,1.001186,1.002032,200,100,76,2
...,...,...,...,...,...,...,...,...,...,...
1514768,32767,587,1.000113,1.000373,1.000081,1.000535,125,200,185,80
1514769,32767,590,1.000113,1.000373,1.000081,1.000405,125,100,214,100
1514770,32767,596,1.000146,1.000405,1.000081,1.000535,205,100,314,78
1514771,32767,597,1.000146,1.000502,1.000081,1.000535,205,125,314,78
